# Machine Learning Engineer Nanodegree
## Capstone Project
### Credit Card Fraud Detection
Xueqiu Feng
June 27th, 2018

## I. Definition

### Project Overview

- #### Problem Domain
Wikipedia defines fraud as: *deliberate deception to secure unfair or unlawful gain, or to deprive a victim of a legal right*. Fraud or deliberate deception is a skill that every species has already mastered to perfection through evolution, which is especially true for human being. In the modern era, with the development of new technologies, new forms of fraud have also been invented. Although fraud appears to be rarely happening, it can result in an huge amount of loss. Therefore, fraud detection is a focal point for insurance, financial, retail and tele-communication sectors. It also has drawed attention from the academical 

- #### Project Origin
I used the Kaggle dataset [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud), which was collected and analysed during a research collaboration of Worldline and the [Machine Learning Group of ULB (Université Libre de Bruxelles)](http://mlg.ulb.ac.be) on big data mining and fraud detection. More details on current and past projects on related topics are available on http://mlg.ulb.ac.be/BruFence and http://mlg.ulb.ac.be/ARTML.

- #### Dataset
The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions. It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features $V_1, V_2, ... V_{28}$ are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

### Problem Statement

Credit card is almost a necessity to survive in the modern society. However, it associates with risk exposure because of physical or informational theft. Whereas a stolen card can be reported and frozen immediately to prevent unauthorised transactions, a compromised account can be abused without notice untill receiving bill statement, where before the frudulent use the account information can be hold for an arbitrary time, making the source even harder to trace. Because of the huge amount of transactions, it is nearly impossible to check them one by one manuelly, which will also inevitablly result in inacceptable delay of transactions. 

It is important that, credit card companies are able to recognize fraudulent credit card transactions, so that customers are not charged for items that they did not purchase. Therefore it would be very meaningful to build a system which can automatically detect dubious transactions and freeze it for further inspection.

The creditcard fraud detection problem can be easily formulated as a supervised machine learning problem, with the binary labels *fraud* or *non-fraud*, i.e. binary classification. Therefore almost all the possible supervised machine learning algorithms can be applied. Where the *AUC* and *F score* can be very relevant in terms of determining the quality of the classifier, for the reason that the dataset is strongly imbalanced.

Neural network will not be considered because of the long training time and the difficulty of finding the best architecture. Moreover, my goal is to use the conventional machine learning algorithms to come as close as possible to be competitive against neural network, which will also be my benchmark model. At the end the classifiers will be compared such that the optimal one, depending on my own definition of optimality, will be chosen to tackle this particular problem.

### Metrics

Due to the imbalanced nature of this dataset and the binary of the classification itself, following metrics are applied:

- #### F Score

It is defined as 
$$
\begin{equation}
F_{\beta} = (1+\beta^2)\frac{precisision \cdot recall}{\beta^2\cdot precision + recall}
\end{equation}, \quad\beta\in [0,+\infty)
$$
where
$$
\begin{equation}
precision = \frac{tp}{tp + fp}
\end{equation}
$$
i.e. out all all the transactions which are classified as fraud, the share of real fraud transactions.
$$
\begin{equation}
recall = \frac{tp}{tp + fn}
\end{equation}
$$
i.e. the success rate of detection if a fraud transaction is being conducted.

For our dataset, where the positive (frauds) class only account for 0.172% of all transactions, the measure of *recall* shall be much more important than *precision*. Because it would be preferable to identify non-fraud as fraud then the other way around, i.e. it might be sensible to pay the price that some of the non-fraud transactions are classified as fraud, in order to identify all the fraud transactions.

However, in an extreme scenario where we assert indiscriminately that, all the samples are fraud, *recall* will be a perfect 100%, whereas *precision* will be nearly 0.

Thus we will use the $F_{0.5}$ score to give *recall* more weight over *precision* in the evaluation.

- #### Recall & Precision

Alternatively we could also simply use *recall* and *precision* as metric.

- #### Area under the Receiver Operating Characteristic Curve (AUROC)

A *receiver operating characteristic curve*, i.e. *ROC curve*, is created by plotting *recall* against *false positive rate (fpr)*, which is defined as
$$
\begin{equation}
fpr = \frac{fp}{fp + tn}
\end{equation}
$$
i.e. the ratio between the number of normal transactions wrongly categorized as fraud and the total number of actual normal transactions, with thresholds from 0 to 1.

Which can be understood as how the classifier's performance varies with different thresholds, where the classifier has an output of probability predicting a transaction being fraud. And only if the probability passes the threshold, the transaction shall be categorized as fraud. Notice that this measure is therefore only suitable for classifiers that have probablistic outputs.

The area under the *ROC curve* (often referred to as simply the *AUC* or *AUROC*) is equal to the probability that a classifier will rank a randomly chosen positive instance higher than a randomly chosen negative one (assuming 'positive' ranks higher than 'negative')<cite data-cite="5955268/AL6LRDFR"></cite>.

- #### False positive Rate

False positive rate is also an useful metric, it indicates what is the proportion of normal transactions being classified as fraudulent. Since there is an huge number of transactions occur every minute, it would be undesirable to constantly freeze normal transactions for further inspections.


##### Among the above mentioned metrics, I consider *AUC*, *recall* and *false positive rate* the most relevent metrics for evaluation the performance of classifiers for this problem.

## II. Analysis

### Data Exploration

- #### Original Data
![](data.png)

- #### Descriptive Statistics
![](data_describe.png)
Obviously, the 28 anonamised features are centered already. Whereas the feature 'Amount' is not only not centered, but has also a totally different scale, which means I have some data pre-processing work to do later on.

- #### Missing Values
![](na.png)
There are no missing values in every feature, i.e. 'Time', V1, ..., V29, 'Amount' are missing value free.

### Exploratory Visualization

- #### Time
![](tt.png)
We observe that, the majority of transactions (non-fraud) were conducted periodically. There is also a vage pattern of fraud transactions, presumly occured during the night.

- #### Amount
![](aa.png)
Clearly, the amounts need to be transformed such that there is more discrimination.

### Algorithms and Techniques
I aim to use simple and interpretable algorithms to tackle the problem of fraud detection, where at the end there are probability outputs of either class. Thus in my opion the following algorithms are potential candidates.

- #### Decision Tree
This is the most intuitive algorithm, although there are some very technical details about how the tree should grow exactly. Basically it is just as the same as our daily decision makings, where we ask ourselves a sequence of simple questions which can be answered with 'yes' or 'no'. At the end we will reach a decision depending on our answers.

The algorithm was implemented with the default settings from sklearn except that the *random_state* is set to 0, i.e. *(criterion=’gini’, splitter=’best’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=0, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)*

- #### Gaussian Naive Bayes
Gaussian naive Bayes is a popular algorithm in classification problems with approximately Gaussian features, which is our case after some transformations. I will give it a chance and see if it can handle this job well. Generally, we assume that given the class, the features are independently Gaussian distributed. Thus given features $\mathbf{x} = (x_1, ..., x_p)$ for an arbitrary class $\mathbf{y}$ we have: 

$$
\begin{align}
\mathbb{P}(\mathbf{y} \lvert \mathbf{x}) &= \frac{\mathbb{P}(\mathbf{y})\mathbb{P}(\mathbf{x} \lvert \mathbf{y})}{\mathbb{P}(\mathbf{x})} \\
&= \frac{\mathbb{P}(\mathbf{y})\prod_{i=1}^p\mathbb{P}(x_i\lvert \mathbf{y})}{\mathbb{P}(\mathbf{x})} \propto \mathbb{P}(\mathbf{y})\prod_{i=1}^p\frac{1}{\sqrt{2\pi\sigma_\mathbf{y}^2}}exp\left(-\frac{(x_i-\mu_\mathbf{y})^2}{2\sigma_\mathbf{y}^2}\right)
\end{align}
$$
where the parameters $\sigma_\mathbf{y}$ and $\mu_\mathbf{y}$ are estimated using maximum likelihood.

The optimal classification $\hat{\mathbf{y}}$ is determined by the maximum likelihood estimator:
$$
\begin{equation}
\hat{\mathbf{y}} = \underset{\mathbf{y}}{\arg\max}\mathbb{P}(\mathbf{y})\prod_{i=1}^p\frac{1}{\sqrt{2\pi\sigma_\mathbf{y}^2}}exp\left(-\frac{(x_i-\mu_\mathbf{y})^2}{2\sigma_\mathbf{y}^2}\right)
\end{equation}
$$

The *prior* $\mathbb{P}(\mathbf{y})$ can either be specified or automatically adjusted according to the heuristic distribution probability of each class in the training data. In my project the latter and default one in *sklearn* was adopted.

- #### Logistic Regression
Logistic regression with L2 penalty and the hyperparameter $C\in(0, +\infty)$ which controls how important the penalty should be, can be formulated as the following optimization problem:
$$
\begin{equation}
\underset{w, b}{\min}\frac{1}{2}ww^T + C\sum_{i=1}^nlog(exp(-y_i(X_iw^T + b)) + 1)
\end{equation}
$$
where $X_1,..., X_n$ are X training samples.

The probability that a certain sample $X_k$ will be classified as fraud is:
$$
\begin{equation}
\mathbb{P}(Y_k = 1) = \frac{1}{1 + e^{-X_kw^T - b}}
\end{equation}
$$

The algorithm is implemented with the default parameters from sklearn with the exception that *random_state* is set to 0, i.e. *(penalty=’l2’, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=0, solver=’liblinear’, max_iter=100, multi_class=’ovr’, verbose=0, warm_start=False, n_jobs=1)*
### Benchmark

As mentioned before, I plan to benchmark my model against a neural network, which is trained by [Currie32 on Kaggle](https://www.kaggle.com/currie32/predicting-fraud-with-tensorflow) for exactly the same dataset. He claimed to have achieved a *recall* of 82.93% and a false positive rate of 0.10%, i.e. 82.92% of all fraudulent transitions were detected and 0.10% of the non-fraud transactions were falsely reported as fraud.

## III. Methodology

### Data Preprocessing
As already mentioned above, there is some pre-processing needed since the features 'Time' and 'Amount' are of much bigger scale than V1, ..., V28.

- #### Time

Clearly, the original feature 'Time' is the time eslapsed in second since the first transaction. Thus we can take the modulo 24x60x60 = 86,400, i.e. the total seconds in a day, to make it periodic. The result after the transformation is
![](time_mod.png)

It then needs to be rescaled using logarithmization and then normalization, which results in 
![](time_trans.png)

The transformed feature 'Time' can now be used for discriminate between *fraud* and *non-fraud*, since they have very different distribution.

- #### Amount
The feature 'Amount' after logarithmization and normalization has the following distribution:
![](amount_trans.png)

### Implementation

- #### Shuffle and Split
    - Normal Dataset
    
    The dataset is splited with 80% training and 20% testing data. In order to make the results reproducible, a random state was set.
    ![](splitting.png)

    - Undersampled Dataset
    
    Since the dataset is highly imbalanced, so is the training set, i.e. in the training data there is only a fraction of samples which are fraudulent. Which will presumably result in very low *recall* because the classifiers would not be 'well-educated' enough for identifying frauds. The easiest and most efficient way to tackle this problem is the so-called *undersampling*, i.e. the training set will be sampled in a way such that there are 50% *fraud* and 50% *non-fraud* training samples.
    ![](undersample.png)
    Thus 80% of the fraudulent transactions and the same amount of normal transactions, i.e. 789 samples for each class, will be our training set in the undersampling trial. The testing set which is created here will not be used for the purpose of testing. The classifiers trained on this undersampled training set will be tested on the same testing set from the *normal dataset* above.
    
- #### Pipeline
I wrote a pipeline, such that I can train different classifiers and obtain the performance metrics on both the training and testing set in an on-line-code manner. The function has the inputs *classifier*, *training features*, *training labels*, *testing features* and *testing labels*. It starts with
![](pipeline1.png)
such that we can document the training and prediction time of the classifier. Then it calculates the performance metrics without AUC on the training set
![](pipeline2.png)
thereafter the performance metrics, all the *fpr* and *fpr* with different thresholds for the *ROC curve* and the *AUC* on the testing set
![](pipeline3.png)
at the end the metrics and the *ROC* as output
![](pipeline4.png)

### Refinement
In this section, you will need to discuss the process of improvement you made upon the algorithms and techniques you used in your implementation. For example, adjusting parameters for certain models to acquire improved solutions would fall under the refinement category. Your initial and final solutions should be reported, as well as any significant intermediate results as necessary. Questions to ask yourself when writing this section:
- _Has an initial solution been found and clearly reported?_
- _Is the process of improvement clearly documented, such as what techniques were used?_
- _Are intermediate and final solutions clearly reported as the process is improved?_


## IV. Results

### Model Evaluation
- #### Normal Dataset
    The 3 different classifiers are trained on the normal training set and testing on the normal testing set.
    ![](normal_train.png)
    Results are
    ![](result_normal.png)
    - ##### Decision Tree
    We observe that, *decision tree classifier* has been highly overfitted and is the most computationally expensive. It has only achieve a *recall* score of 0.8119 and an *AUROC* of 0.9057. Its *false positive rate* is 0.0004, which is decent but not the best.
    
    - ##### Gaussian Naive Bayes
    Gaussian naive Bayes offers the best *recall* score of 0.8515, with the price that the *false positive rate* being the highest, which is 0.0214.
    
    - ##### Logistic Regression
    Logistic Regression has the best performance on the testing set regarding *AUROC* and *false positive rate* with the scores of 0.9783 and 0.0001 respectively. Unfortunately, the *recall* score is only as low as 0.6337. Which means only 63.37% of the fraudulent transactions were successfully detected.

- #### Undersampled Dataset
    The same classifiers are trained on the undersampled training set but testing on the normal testing set.
    ![](under_train.png)
    with the results
    ![](result_under.png)
    - ##### Decision Tree
    This time *decision tree classifier* is still overfitted, however, it has achieve the best *recall* score of 0.9901 on the testing set. On the other hand, it has the worst performance on the testing set in terms of *false positive rate* and *AUROC*, 0.1057 and 0.9422 respectively, meaning that it has identified most frauds by paying the price of falsely categorizing most normal transactions.
    
    - ##### Gaussian Naive Bayes
    Gaussian naive Bayes has achieved the best *false positive rate* of 0.0381 but also the worst *recall* score of 0.8713 on the testing set. It has an *AUROC* of 0.9618.
    
    - ##### Logistic Regression
    Logistic regression performanced best on the testing set regarding *AUROC* with 0.9837. At the same time, its *false positive rate* is 0.0399, only a tick higher than that of Gaussian naive Bayes, while achieving a *recall* score of 0.9505.
    
- #### The Optimal Solution
    After considering all the metrics comprehensively, I concluded that *logistic regression* with *undersampling* has the best performance on this particular problem.
    
### Model Validation
Now I would like to validate *logistic regression* with 5 different *undersampled training sets* to see if it can be generalized. It is tested on the same normal testing set from before.
![](cv.png)
The results are:
![](cv_result.png)
It confirms that *logistic regression* offers very stable performances with different *undersampled training sets*.

### Justification

Comparing to the benchmark results, my model offers better *recall* with around 95.25% against 82.92%. However, it has much worse score on *false positive rate* with around 3.27% against 0.10%. I would consider this even with the benchmark results, because although *recall* is improved from 82.92% to 95.25%, the *false positive rate* increased more than 300%. Depending on how important *recall* is, we can now therefore adapt different strategies.

## V. Conclusion

### Reflection

In my opinion, the most interesting aspect of this project is that we can even achieve better results with less training data because of the skewness in the normal training set. With this project, I am convinced that *undersampling* is indeed an useful technique to deal with imbalanced classification problem.

It is also very important that in the data exploration, we should consider which features need to be transformed such that it makes more sense for the algorithms.

The major difficulty of this project is defining the metrics in a meaningful way. Some metrics, e.g. *accuracy* would not be suitable for imbalanced classification problem. While *AUROC* and *recall* would make more sense.

### Improvement
There is an improvement potential if we consider the feature 'Amount' not as a normal feature just like the other, but as a penalty for the mis-classification in the training phase. To be more concrete, we could use the untransformed original amount $\alpha\in[0,+\inf]$ to re-define the optimization problem of *logistic regression* as:
$$
\begin{equation}
\underset{w, b}{\min}\frac{1}{2}ww^T + C\sum_{i=1}^nlog(exp(-\alpha_i\cdot d(y_i, X_iw^T + b)) + 1)
\end{equation}
$$
i.e. transactions with bigger amount will be weighted more heavily, where $d(.,.)$ is some distance metrics.

Another possibility would be that we fine tune the hyperparameter $C$ using cross validation. Or even we can try different optimization strategies to solve the above problem.

### Bibliography
<div class="cite2c-biblio"></div>